In [2]:
import nltk,os
import requests
from libs import Stanford
from libs import TweetRepository
import pyprind
nltk.__version__

'3.1'

Se invoca a la API Rest para el control del servidor NER.

In [5]:
baseUrl = 'http://ner-server-1:5000'

In [6]:
url = baseUrl + '/ner/models'
response = requests.get(url)
print 'Modelos NER disponibles',response.content

ConnectionError: HTTPConnectionPool(host='ner-server-1', port=5000): Max retries exceeded with url: /ner/models (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x7f1f0147ff90>: Failed to establish a new connection: [Errno -2] Name or service not known',))

In [ ]:
url = baseUrl + '/ner/start/spanish.ancora.distsim.s512.crf.ser.gz'
response = requests.get(url)
print 'Iniciar',response.content

In [ ]:
url = baseUrl + '/ner/status'
response = requests.get(url)
print 'Status NER',response.content

In [ ]:
url = baseUrl + '/pos/start'
response = requests.get(url)
print 'Iniciar POS',response.content

In [ ]:
url = baseUrl + '/pos/status'
response = requests.get(url)
print 'Status POS',response.content

Se invoca el cliente para la comunicación con el servidor NER a travez de Sockets.
Se invoca el cliente para la comunicación con la colección de tweets-

In [ ]:
stanford_client = Stanford(baseUrl.split('//')[1].split(':')[0])
repositorio_tweets = TweetRepository.TweetRepositoryNER('imported-tweets-ner')

Usando el POS Tagger y el NER tagger, se le asigna a cada palabra una clase candidata. Se asigna LABEL cuando es un nombre propio que no se puede clasificar.

In [ ]:
todos = repositorio_tweets.getAll()
bar = pyprind.ProgBar(len(todos))
palabras_anotadas = []
for tweet in todos:
    for par in stanford_client.entities(tweet['text']):
        #Tengo que anexar el id para despues
        l = list(par)
        l.insert(2,tweet['_id'])
        t = tuple(l)
        palabras_anotadas.append(t)
    bar.update()

In [ ]:
print 'Cantidad de palabras:',len(palabras_anotadas)

Usando listas de calles y ubicaciones conocidas de la ciudad se buscan las LABEL que aparezcan en las listas y se sustituye la etiqueta.

In [ ]:
dicc_loc = stanford_client.loadStreetNames('v_mdg_nombres_vias_mvd.txt')

In [ ]:
dicc_loc = stanford_client.loadGeoPlaces('UY.txt',dicc_loc)

In [ ]:
palabras_anotadas_mejoradas = stanford_client.replaceLabelsUsingLists(palabras_anotadas,dicc_loc)

Se actualizan en la base los tweets con las entidades.

In [ ]:
tweets_modificados = repositorio_tweets.updateTweetsWithEntities(palabras_anotadas_mejoradas)

In [ ]:
import pprint
for t in tweets_modificados:
    if len(t['entities'].keys()) > 5:
        print '_'
        for e in t['entities']:
            if e in ['PERS','LUG','ORG','OTROS']:
                print t['entities'][e]

Se generan los archivos de train y test para NER usando lo generado antes

In [ ]:
stanford_client.generateTrainAndTestFile(palabras_anotadas_mejoradas,'prueba_train.tsv','prueba_test.tsv',1)

Se transfiere el archivo usando la API

In [ ]:
url = baseUrl + '/ner/train'
files = {'file': open('prueba_train.tsv')}
response = requests.post(url, files=files)
print response.content

Se entrena el modelo

In [ ]:
url = baseUrl + '/ner/train'
response = requests.get(url)

In [ ]:
print 'Salida',response.content[1]

In [ ]:
url = baseUrl + '/ner/models'
response = requests.get(url)
print response.content